In [1]:
import numpy as np
import pandas as pd
import ast
from keras.utils import register_keras_serializable
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPooling2D, Reshape
from keras.saving import load_model
from tensorflow import keras as tfk
import tensorflow as tf
import gc

2024-08-30 10:39:15.811990: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 10:39:15.814330: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 10:39:15.842213: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 10:39:16.422398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow.keras.datasets import mnist
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
dataa = np.load('../data/forr_rand_fns_n' + '10' + '.npz')

In [9]:
dataa['rand_f']

array([[[-1.,  1., -1., ...,  1.,  1., -1.],
        [ 1.,  1., -1., ..., -1.,  1., -1.]],

       [[-1., -1., -1., ..., -1.,  1., -1.],
        [ 1.,  1.,  1., ...,  1.,  1., -1.]],

       [[-1.,  1., -1., ..., -1., -1., -1.],
        [ 1., -1., -1., ...,  1.,  1.,  1.]],

       ...,

       [[-1., -1.,  1., ...,  1., -1., -1.],
        [-1.,  1.,  1., ..., -1., -1.,  1.]],

       [[ 1.,  1.,  1., ..., -1.,  1.,  1.],
        [-1.,  1.,  1., ...,  1., -1., -1.]],

       [[-1., -1., -1., ...,  1.,  1., -1.],
        [ 1., -1.,  1., ..., -1.,  1.,  1.]]])

n_str = 4 # no polling because of dimensionality

    x = Reshape((img_size, img_size, 1))(input)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(int(N/4), activation='relu')(x)
    # x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    # x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)

n_str = 6, 8, 10, 12

    x = Reshape((img_size, img_size, 1))(input)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(int(N/4), activation='relu')(x)
    # x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    # x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)

In [3]:
# Set n
n_strs = ['4']

@register_keras_serializable()
def euclidean_distance(vects):
    """
    Calculates the Euclidean distance between two vectors.

    Args:
        vects: A tuple of two vectors (x, y).

    Returns:
        The Euclidean distance between the two vectors.
    """
    x, y = vects
    return tfk.backend.sqrt(tfk.backend.maximum(tfk.backend.sum(tfk.backend.square(x - y), axis=1, keepdims=True), tfk.backend.epsilon()))

@register_keras_serializable()
def cosine_distance(vects):
    """
    Calculates the cosine similarity between two vectors.

    Args:
        vects: A tuple of two vectors (x, y).

    Returns:
        The cosine similarity between the two vectors.
    """
    x, y = vects
    dot_product = tfk.backend.sum(x * y, axis=1, keepdims=True)
    norm_x = tfk.backend.sqrt(tfk.backend.sum(tfk.backend.square(x), axis=1, keepdims=True))
    norm_y = tfk.backend.sqrt(tfk.backend.sum(tfk.backend.square(y), axis=1, keepdims=True))
    return dot_product / (norm_x * norm_y + tfk.backend.epsilon())

@register_keras_serializable()
def eucl_dist_output_shape(shapes):
    """
    Calculates the output shape of the Euclidean distance layer.

    Parameters:
    shapes (tuple): A tuple containing the shapes of the input tensors.

    Returns:
    tuple: A tuple representing the output shape of the Euclidean distance layer.
        The first dimension is the batch size, and the second dimension is 1.
    """
    shape1, shape2 = shapes
    return (shape1[0], 1)

def create_base_network(input_shape, N):
    """
    Creates a base network for a neural network model.

    Parameters:
    input_shape (tuple): The shape of the input data.

    Returns:
    keras.Model: The base network model.
    """
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(N, activation='relu')(x) # use activation='sigmoid' if gradient-based optimization gives problems
    # x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    # x = Dropout(0.1)(x)
    # x = Dense(N, activation='relu')(x)
    # # x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    return Model(input, x)

def create_conv_base_network(input_shape, N):
    """
    Creates a convolutional base network for a neural network model that processes
    a 2D image of size sqrt(N) by sqrt(N) from an input vector of length N.

    Parameters:
    input_shape (tuple): The shape of the input data, which should be (N,).
    N (int): The length of the input vector, which should be a perfect square.

    Returns:
    keras.Model: The base network model with convolutional layers.
    """
    assert np.sqrt(N) % 1 == 0, "N must be a perfect square"
    img_size = int(np.sqrt(N))
    input = Input(shape=input_shape)
    x = Reshape((img_size, img_size, 1))(input)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    return Model(input, x)

for n_str in n_strs:

    # Set the random seed for reproducibility
    np.random.seed(1)

    ########################################################
    # Load data
    file_root = '../data/forrelation/'
    data = np.load(file_root + 'forr_rand_fns_n' + n_str + '.npz')
    # data = np.load(file_root + 'forr_rand_fns_n' + n_str + '_change_basis.npz')
    ########################################################

    N = data['forr_f'].shape[2]

    # Load indices for training set samples
    df = pd.read_csv(file_root + 'tr_inds.csv', header=None)
    tr_idx = df.values
    # Load indices for test set samples
    df = pd.read_csv(file_root + 'te_inds.csv', header=None)
    te_idx = df.values

    ########################################################
    # Create the base network
    base_network = create_base_network((N,), N) # NN
    # base_network = create_conv_base_network((N,), N) # CNN
    ########################################################

    # Create the left input and point to the base network
    input_a = Input(shape=(N,))
    vect_output_a = base_network(input_a)

    # Create the right input and point to the base network
    input_b = Input(shape=(N,))
    vect_output_b = base_network(input_b)

    ########################################################
    # Measure the similarity of the two vector outputs
    # distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])
    # output = Dense(1, activation="sigmoid")(distance)
    output = Lambda(cosine_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])
    ########################################################

    # Specify the inputs and output of the model
    model = Model([input_a, input_b], output)
    
    ########################################################
    # Compile the model
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
    # model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    ########################################################

    # Save the weights of the model for easier resetting
    # model.save_weights('model.weights.h5', overwrite=True)
    # Save the model
    # model.save('model.keras', overwrite=True)
    model.save('model'+str(n_str)+'.keras', overwrite=True, include_optimizer=False)

    # Create result list whose 8 columns are:
    # train loss (mean, std), test loss (mean, std), train acc (mean, std), test acc (mean, std)
    results_raw = np.zeros((50, 10, 4))
    results = np.zeros((10, 8))

    # Create training data + averages
    n_samples, n_avg = tr_idx.shape
    for i in range(n_samples):
        print("##### New sample: " + str(i+1) + " #####")
        for j in range(n_avg):
            print("----------------  Sample: " + str(i+1) + " | Average: " + str(j+1) + "  ----------------")
            # Load initial weights
            # model.load_weights('model.weights.h5')
            # Load model
            model = load_model('model'+str(n_str)+'.keras', compile=False)
            # model = Model([input_a, input_b], output)
            model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
            # Prepare a test indices with 40 samples that are not in the training set with no repetitions
            # test_idx = np.random.choice([i for i in range(50) if i not in train_idx], 40, replace=False)
            # np.all(np.sort(ast.literal_eval(test_idx[0, 0])) == np.unique(ast.literal_eval(test_idx[0, 0])))
            # Training data
            train_idx = ast.literal_eval(tr_idx[i, j])
            x_train = np.zeros(((i+1)*2, 2, N))
            y_train = np.zeros((i+1)*2)
            for k in range(len(train_idx)):
                x_train[k, :, :] = data["forr_f"][train_idx[k], :, :]
                y_train[k] = 0
                x_train[k+(i+1), :, :] = data["rand_f"][train_idx[k], :, :]
                y_train[k+(i+1)] = 1
            # Test data
            test_idx = ast.literal_eval(te_idx[i, j])
            x_test = np.zeros((80, 2, N))
            y_test = np.zeros(80)
            for k in range(40):
                x_test[k, :, :] = data["forr_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k] = 0
                x_test[k+40, :, :] = data["rand_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k+40] = 1
            # Reshape the data to fit the network input (samples, vector)
            x_train_1 = x_train[:, 0].reshape(-1, N)
            x_train_2 = x_train[:, 1].reshape(-1, N)
            x_test_1 = x_test[:, 0].reshape(-1, N)
            x_test_2 = x_test[:, 1].reshape(-1, N)

            # Train the model and store the history into results_raw
            history = model.fit([x_train_1, x_train_2], y_train, batch_size=1, epochs=300, validation_data=([x_test_1, x_test_2], y_test), verbose=0)
            results_raw[j, i, :] = [history.history['loss'][-1], history.history['val_loss'][-1], history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
            del history
            del model
            gc.collect()

    # Calculate the averages and standard deviations
    for i in range(10):
        results[i, 0::2] = np.mean(results_raw[:, i, :], axis=0)
        results[i, 1::2] = np.std(results_raw[:, i, :], axis=0)

    # np.savez(file_root + 'forr_siamese_dnn_n' + n_str + '.npz', results=results)
    
    # np.savez(file_root + 'forr_siamese_nn_n' + n_str + '_bce.npz', results=results)
    np.savez(file_root + 'forr_siamese_nn_n' + n_str + '_cosine_bce.npz', results=results)
    # np.savez(file_root + 'forr_siamese_nn_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_nn_drop_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_nn_drop_n' + n_str + '_probability001.npz', results=results)
    # np.savez(file_root + 'forr_siamese_nn_cb_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_nn_cb_drop_n' + n_str + '.npz', results=results)
    
    # np.savez(file_root + 'forr_siamese_cnn_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_cnn_drop_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_cnn_cb_n' + n_str + '.npz', results=results)
    # np.savez(file_root + 'forr_siamese_cnn_cb_drop_n' + n_str + '.npz', results=results)

##### New sample: 1 #####
----------------  Sample: 1 | Average: 1  ----------------
----------------  Sample: 1 | Average: 2  ----------------
----------------  Sample: 1 | Average: 3  ----------------
----------------  Sample: 1 | Average: 4  ----------------
----------------  Sample: 1 | Average: 5  ----------------
----------------  Sample: 1 | Average: 6  ----------------
----------------  Sample: 1 | Average: 7  ----------------
----------------  Sample: 1 | Average: 8  ----------------
----------------  Sample: 1 | Average: 9  ----------------
----------------  Sample: 1 | Average: 10  ----------------
----------------  Sample: 1 | Average: 11  ----------------
----------------  Sample: 1 | Average: 12  ----------------
----------------  Sample: 1 | Average: 13  ----------------
----------------  Sample: 1 | Average: 14  ----------------
----------------  Sample: 1 | Average: 15  ----------------
----------------  Sample: 1 | Average: 16  ----------------
----------------  Sampl

In [363]:
# Set n
n_strs = ['6', '8', '10']

@register_keras_serializable()
def euclidean_distance(vects):
    """
    Calculates the Euclidean distance between two vectors.

    Args:
        vects: A tuple of two vectors (x, y).

    Returns:
        The Euclidean distance between the two vectors.
    """
    x, y = vects
    return tfk.backend.sqrt(tfk.backend.maximum(tfk.backend.sum(tfk.backend.square(x - y), axis=1, keepdims=True), tfk.backend.epsilon()))

@register_keras_serializable()
def eucl_dist_output_shape(shapes):
    """
    Calculates the output shape of the Euclidean distance layer.

    Parameters:
    shapes (tuple): A tuple containing the shapes of the input tensors.

    Returns:
    tuple: A tuple representing the output shape of the Euclidean distance layer.
        The first dimension is the batch size, and the second dimension is 1.
    """
    shape1, shape2 = shapes
    return (shape1[0], 1)

def create_base_network(input_shape, N):
    """
    Creates a base network for a neural network model.

    Parameters:
    input_shape (tuple): The shape of the input data.

    Returns:
    keras.Model: The base network model.
    """
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(N, activation='relu')(x) # use activation='sigmoid' if gradient-based optimization gives problems
    # x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    # x = Dropout(0.1)(x)
    # x = Dense(N, activation='relu')(x)
    # # x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    return Model(input, x)

def create_conv_base_network(input_shape, N):
    """
    Creates a convolutional base network for a neural network model that processes
    a 2D image of size sqrt(N) by sqrt(N) from an input vector of length N.

    Parameters:
    input_shape (tuple): The shape of the input data, which should be (N,).
    N (int): The length of the input vector, which should be a perfect square.

    Returns:
    keras.Model: The base network model with convolutional layers.
    """
    assert np.sqrt(N) % 1 == 0, "N must be a perfect square"
    img_size = int(np.sqrt(N))
    input = Input(shape=input_shape)
    x = Reshape((img_size, img_size, 1))(input)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    return Model(input, x)

for n_str in n_strs:

    # Set the random seed for reproducibility
    np.random.seed(1)

    ########################################################
    # Load data
    data = np.load('../data/forr_rand_fns_n' + n_str + '.npz')
    # data = np.load('../data/forr_rand_fns_n' + n_str + '_change_basis.npz')
    ########################################################

    N = data['forr_f'].shape[2]

    # Load indices for training set samples
    df = pd.read_csv('../data/tr_inds.csv', header=None)
    tr_idx = df.values
    # Load indices for test set samples
    df = pd.read_csv('../data/te_inds.csv', header=None)
    te_idx = df.values

    ########################################################
    # Create the base network
    base_network = create_base_network((N,), N) # NN
    # base_network = create_conv_base_network((N,), N) # CNN
    ########################################################

    # Create the left input and point to the base network
    input_a = Input(shape=(N,))
    vect_output_a = base_network(input_a)

    # Create the right input and point to the base network
    input_b = Input(shape=(N,))
    vect_output_b = base_network(input_b)

    # Measure the similarity of the two vector outputs
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])
    output = Dense(1, activation="sigmoid")(distance)

    # Specify the inputs and output of the model
    model = Model([input_a, input_b], output)
    
    ########################################################
    # Compile the model
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
    # model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    ########################################################

    # Save the weights of the model for easier resetting
    # model.save_weights('model.weights.h5', overwrite=True)
    # Save the model
    # model.save('model.keras', overwrite=True)
    model.save('model'+str(n_str)+'.keras', overwrite=True, include_optimizer=False)

    # Create result list whose 8 columns are:
    # train loss (mean, std), test loss (mean, std), train acc (mean, std), test acc (mean, std)
    results_raw = np.zeros((50, 10, 4))
    results = np.zeros((10, 8))

    # Create training data + averages
    n_samples, n_avg = tr_idx.shape
    for i in range(n_samples):
        print("##### New sample: " + str(i+1) + " #####")
        for j in range(n_avg):
            print("----------------  Sample: " + str(i+1) + " | Average: " + str(j+1) + "  ----------------")
            # Load initial weights
            # model.load_weights('model.weights.h5')
            # Load model
            model = load_model('model'+str(n_str)+'.keras', compile=False)
            # model = Model([input_a, input_b], output)
            model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
            # Prepare a test indices with 40 samples that are not in the training set with no repetitions
            # test_idx = np.random.choice([i for i in range(50) if i not in train_idx], 40, replace=False)
            # np.all(np.sort(ast.literal_eval(test_idx[0, 0])) == np.unique(ast.literal_eval(test_idx[0, 0])))
            # Training data
            train_idx = ast.literal_eval(tr_idx[i, j])
            x_train = np.zeros(((i+1)*2, 2, N))
            y_train = np.zeros((i+1)*2)
            for k in range(len(train_idx)):
                x_train[k, :, :] = data["forr_f"][train_idx[k], :, :]
                y_train[k] = 0
                x_train[k+(i+1), :, :] = data["rand_f"][train_idx[k], :, :]
                y_train[k+(i+1)] = 1
            # Test data
            test_idx = ast.literal_eval(te_idx[i, j])
            x_test = np.zeros((80, 2, N))
            y_test = np.zeros(80)
            for k in range(40):
                x_test[k, :, :] = data["forr_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k] = 0
                x_test[k+40, :, :] = data["rand_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k+40] = 1
            # Reshape the data to fit the network input (samples, vector)
            x_train_1 = x_train[:, 0].reshape(-1, N)
            x_train_2 = x_train[:, 1].reshape(-1, N)
            x_test_1 = x_test[:, 0].reshape(-1, N)
            x_test_2 = x_test[:, 1].reshape(-1, N)

            # Train the model and store the history into results_raw
            history = model.fit([x_train_1, x_train_2], y_train, batch_size=128, epochs=300, validation_data=([x_test_1, x_test_2], y_test), verbose=0)
            results_raw[j, i, :] = [history.history['loss'][-1], history.history['val_loss'][-1], history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
            del history
            del model
            gc.collect()

    # Calculate the averages and standard deviations
    for i in range(10):
        results[i, 0::2] = np.mean(results_raw[:, i, :], axis=0)
        results[i, 1::2] = np.std(results_raw[:, i, :], axis=0)

    np.savez('../data/forr_siamese_nn_n' + n_str + '_binarycrossentropy.npz', results=results)

    # np.savez('../data/forr_siamese_dnn_n' + n_str + '.npz', results=results)

    # np.savez('../data/forr_siamese_nn_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_drop_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_drop_n' + n_str + '_probability001.npz', results=results)
    # np.savez('../data/forr_siamese_nn_cb_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_cb_drop_n' + n_str + '.npz', results=results)
    
    # np.savez('../data/forr_siamese_cnn_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_cnn_drop_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_cnn_cb_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_cnn_cb_drop_n' + n_str + '.npz', results=results)

##### New sample: 1 #####
----------------  Sample: 1 | Average: 1  ----------------
----------------  Sample: 1 | Average: 2  ----------------
----------------  Sample: 1 | Average: 3  ----------------
----------------  Sample: 1 | Average: 4  ----------------
----------------  Sample: 1 | Average: 5  ----------------
----------------  Sample: 1 | Average: 6  ----------------
----------------  Sample: 1 | Average: 7  ----------------
----------------  Sample: 1 | Average: 8  ----------------
----------------  Sample: 1 | Average: 9  ----------------
----------------  Sample: 1 | Average: 10  ----------------
----------------  Sample: 1 | Average: 11  ----------------
----------------  Sample: 1 | Average: 12  ----------------
----------------  Sample: 1 | Average: 13  ----------------
----------------  Sample: 1 | Average: 14  ----------------
----------------  Sample: 1 | Average: 15  ----------------
----------------  Sample: 1 | Average: 16  ----------------
----------------  Sampl

In [3]:
# Set n
n_strs = ['4']

@register_keras_serializable()
def euclidean_distance(vects):
    """
    Calculates the Euclidean distance between two vectors.

    Args:
        vects: A tuple of two vectors (x, y).

    Returns:
        The Euclidean distance between the two vectors.
    """
    x, y = vects
    return tfk.backend.sqrt(tfk.backend.maximum(tfk.backend.sum(tfk.backend.square(x - y), axis=1, keepdims=True), tfk.backend.epsilon()))

@register_keras_serializable()
def eucl_dist_output_shape(shapes):
    """
    Calculates the output shape of the Euclidean distance layer.

    Parameters:
    shapes (tuple): A tuple containing the shapes of the input tensors.

    Returns:
    tuple: A tuple representing the output shape of the Euclidean distance layer.
        The first dimension is the batch size, and the second dimension is 1.
    """
    shape1, shape2 = shapes
    return (shape1[0], 1)

def create_base_network(input_shape, N):
    """
    Creates a base network for a neural network model.

    Parameters:
    input_shape (tuple): The shape of the input data.

    Returns:
    keras.Model: The base network model.
    """
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(N, activation='relu')(x) # use activation='sigmoid' if gradient-based optimization gives problems
    x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(N, activation='relu')(x)
    return Model(input, x)

def create_conv_base_network(input_shape, N):
    """
    Creates a convolutional base network for a neural network model that processes
    a 2D image of size sqrt(N) by sqrt(N) from an input vector of length N.

    Parameters:
    input_shape (tuple): The shape of the input data, which should be (N,).
    N (int): The length of the input vector, which should be a perfect square.

    Returns:
    keras.Model: The base network model with convolutional layers.
    """
    assert np.sqrt(N) % 1 == 0, "N must be a perfect square"
    img_size = int(np.sqrt(N))
    input = Input(shape=input_shape)
    x = Reshape((img_size, img_size, 1))(input)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(int(N/4), activation='relu')(x)
    return Model(input, x)

for n_str in n_strs:

    # Set the random seed for reproducibility
    np.random.seed(1)

    ########################################################
    # Load data
    data = np.load('../data/forr_rand_fns_n' + n_str + '.npz')
    # data = np.load('../data/forr_rand_fns_n' + n_str + '_change_basis.npz')
    ########################################################

    N = data['forr_f'].shape[2]

    # Load indices for training set samples
    df = pd.read_csv('../data/tr_inds.csv', header=None)
    tr_idx = df.values
    # Load indices for test set samples
    df = pd.read_csv('../data/te_inds.csv', header=None)
    te_idx = df.values

    ########################################################
    # Create the base network
    # base_network = create_base_network((N,), N)
    base_network = create_conv_base_network((N,), N)
    ########################################################

    # Create the left input and point to the base network
    input_a = Input(shape=(N,))
    vect_output_a = base_network(input_a)

    # Create the right input and point to the base network
    input_b = Input(shape=(N,))
    vect_output_b = base_network(input_b)

    # Measure the similarity of the two vector outputs
    output = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])

    # Specify the inputs and output of the model
    model = Model([input_a, input_b], output)

    # Compile the model
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

    # Save the weights of the model for easier resetting
    # model.save_weights('model.weights.h5', overwrite=True)
    # Save the model
    # model.save('model.keras', overwrite=True)
    model.save('model'+str(n_str)+'drop.keras', overwrite=True, include_optimizer=False)

    # Create result list whose 8 columns are:
    # train loss (mean, std), test loss (mean, std), train acc (mean, std), test acc (mean, std)
    results_raw = np.zeros((50, 10, 4))
    results = np.zeros((10, 8))

    # Create training data + averages
    n_samples, n_avg = tr_idx.shape
    for i in range(n_samples):
        print("##### New sample: " + str(i+1) + " #####")
        for j in range(n_avg):
            print("----------------  Sample: " + str(i+1) + " | Average: " + str(j+1) + "  ----------------")
            # Load initial weights
            # model.load_weights('model.weights.h5')
            # Load model
            model = load_model('model'+str(n_str)+'drop.keras')
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
            # Prepare a test indices with 40 samples that are not in the training set with no repetitions
            # test_idx = np.random.choice([i for i in range(50) if i not in train_idx], 40, replace=False)
            # np.all(np.sort(ast.literal_eval(test_idx[0, 0])) == np.unique(ast.literal_eval(test_idx[0, 0])))
            # Training data
            train_idx = ast.literal_eval(tr_idx[i, j])
            x_train = np.zeros(((i+1)*2, 2, N))
            y_train = np.zeros((i+1)*2)
            for k in range(len(train_idx)):
                x_train[k, :, :] = data["forr_f"][train_idx[k], :, :]
                y_train[k] = 1
                x_train[k+(i+1), :, :] = data["rand_f"][train_idx[k], :, :]
                y_train[k+(i+1)] = 0
            # Test data
            test_idx = ast.literal_eval(te_idx[i, j])
            x_test = np.zeros((80, 2, N))
            y_test = np.zeros(80)
            for k in range(40):
                x_test[k, :, :] = data["forr_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k] = 1
                x_test[k+40, :, :] = data["rand_f"][test_idx[k]-1, :, :] # test file has 1-based indexing
                y_test[k+40] = 0       
            # Reshape the data to fit the network input (samples, vector)
            x_train_1 = x_train[:, 0].reshape(-1, N)
            x_train_2 = x_train[:, 1].reshape(-1, N)
            x_test_1 = x_test[:, 0].reshape(-1, N)
            x_test_2 = x_test[:, 1].reshape(-1, N)

            # Train the model and store the history into results_raw
            history = model.fit([x_train_1, x_train_2], y_train, batch_size=128, epochs=300, validation_data=([x_test_1, x_test_2], y_test), verbose=0)
            results_raw[j, i, :] = [history.history['loss'][-1], history.history['val_loss'][-1], history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
            del history
            del model
            gc.collect()

    # Calculate the averages and standard deviations
    for i in range(10):
        results[i, 0::2] = np.mean(results_raw[:, i, :], axis=0)
        results[i, 1::2] = np.std(results_raw[:, i, :], axis=0)

    # np.savez('../data/forr_siamese_nn_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_drop_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_drop_n' + n_str + '_probability001.npz', results=results)
    # np.savez('../data/forr_siamese_nn_cb_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_nn_cb_drop_n' + n_str + '.npz', results=results)
    
    # np.savez('../data/forr_siamese_cnn_n' + n_str + '.npz', results=results)
    np.savez('../data/forr_siamese_cnn_drop_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_cnn_cb_n' + n_str + '.npz', results=results)
    # np.savez('../data/forr_siamese_cnn_cb_drop_n' + n_str + '.npz', results=results)

##### New sample: 1 #####
----------------  Sample: 1 | Average: 1  ----------------


/home/mekise/.local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


----------------  Sample: 1 | Average: 2  ----------------
----------------  Sample: 1 | Average: 3  ----------------
----------------  Sample: 1 | Average: 4  ----------------
----------------  Sample: 1 | Average: 5  ----------------
----------------  Sample: 1 | Average: 6  ----------------
----------------  Sample: 1 | Average: 7  ----------------
----------------  Sample: 1 | Average: 8  ----------------
----------------  Sample: 1 | Average: 9  ----------------
----------------  Sample: 1 | Average: 10  ----------------
----------------  Sample: 1 | Average: 11  ----------------
----------------  Sample: 1 | Average: 12  ----------------
----------------  Sample: 1 | Average: 13  ----------------
----------------  Sample: 1 | Average: 14  ----------------
----------------  Sample: 1 | Average: 15  ----------------
----------------  Sample: 1 | Average: 16  ----------------
----------------  Sample: 1 | Average: 17  ----------------
----------------  Sample: 1 | Average: 18  -----